<a href="https://colab.research.google.com/github/kunjb-sao/Python-ML/blob/main/NLP-Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
temp_df = pd.read_csv('/content/news_data (5).csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/news_data (5).csv'

In [ ]:
df = temp_df.iloc[:10000]

In [ ]:
df.head()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [ ]:
# df['review'] = df['review'].apply(remove_tags)

In [ ]:
df['summary'] = df['summary'].apply(lambda x:x.lower())

In [ ]:
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

sw_list = stopwords.words('english')

df['summary'] = df['summary'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [ ]:
# from nltk.corpus import stopwords

# sw_list = stopwords.words('english')

# df['summary'] = df['summary'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [ ]:
df['summary']

In [ ]:
df

In [ ]:
import gensim

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
story = []
for doc in df['summary']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))


In [ ]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [ ]:
model.build_vocab(story)

In [ ]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
len(model.wv.index_to_key)

In [ ]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [ ]:
document_vector(df['summary'].values[0])

In [ ]:
from tqdm import tqdm

In [ ]:
X = []
for doc in tqdm(df['summary'].values):
    X.append(document_vector(doc))

In [ ]:
X = np.array(X)

In [ ]:
X[0]

In [ ]:
df

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['subject'])

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

In [ ]:
text = "An asteroid nearly the size of a 16-story building is hurtling toward Earth at a staggering speed of 62,650 km/h. Named 2025 DU2, this 160-foot-wide space rock belongs to the Apollo group of asteroids and is set to make a close approach on March 6 at 6:59 AM IST. Although this asteroid will miss Earth by a distance of 542,000 km – which is about 1.4 times the distance between Earth and the Moon – its size and speed make it a stark reminder of the lurking dangers in space."

In [ ]:
text2 = "The Supreme Court has recorded its displeasure with the way certain orders are passed by the Allahabad High Court. A bench headed by justice Abhay Oka on Wednesday disapproved of the way a revision appeal was decided by the high court. Remanding the case back to the Lucknow bench of HC for reconsideration, the top court directed the high court to call for the trial court record before deciding the matter. In January, another bench headed by justice Surya Kant had observed that the functioning of "

In [ ]:
vectorized_text = document_vector(text2)

In [ ]:
rf.predict(vectorized_text.reshape(1,-1))